In [1]:
import os

os.chdir("C:/Users/flexmatch/Desktop/ssom/code/3.SNS-categorizer/src")

In [2]:
from datetime import datetime
import boto3
from io import BytesIO
from dotenv import load_dotenv

import pandas as pd

load_dotenv('C:/Users/flexmatch/Desktop/ssom/code/3.SNS-categorizer/config/.env')
aws_access_key = os.getenv("aws_accessKey")
aws_secret_key = os.getenv("aws_secretKey")
bucket_name = 'flexmatch-data'
region_name = 'ap-northeast-2'

s3 = boto3.client('s3',
                aws_access_key_id=aws_access_key,
                aws_secret_access_key=aws_secret_key,
                region_name=region_name)

today = datetime.now()
year, week, _ = today.isocalendar()

# profile_data_prefix = f'instagram-data/tables/EXTERNAL_2_RECENT_USER_INFO_MTR/year={year}/week={week}'
# media_data_prefix = f'instagram-data/tables/EXTERNAL_2_BY_USER_ID_MEDIA_DTL_INFO/year={year}/week={week}'

profile_data_prefix = f'instagram-data/tables/RECENT_USER_INFO_MTR/year={year}/week={week}/'
media_data_prefix = f'instagram-data/tables/BY_USER_ID_MEDIA_DTL_INFO/year={year}/week={week}/'

# profile 파일 목록
profile_response = s3.list_objects_v2(Bucket=bucket_name, Prefix=profile_data_prefix)
profile_keys = [obj['Key'] for obj in profile_response.get('Contents', []) if obj['Key'].endswith('.parquet')]

# media 파일 목록
media_response = s3.list_objects_v2(Bucket=bucket_name, Prefix=media_data_prefix)
media_keys = [obj['Key'] for obj in media_response.get('Contents', []) if obj['Key'].endswith('.parquet')]

profile_dfs = []

for key in profile_keys:
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    body = obj['Body'].read()
    df = pd.read_parquet(BytesIO(body))
    profile_dfs.append(df)

not_conn_user_info_mtr = pd.concat(profile_dfs, ignore_index=True)

media_dfs = []

for key in media_keys:
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    body = obj['Body'].read()
    df = pd.read_parquet(BytesIO(body))
    media_dfs.append(df)

not_conn_user_media_info = pd.concat(media_dfs, ignore_index=True)

In [113]:
from DB_connection import *
from model_inference import *

In [6]:
# pip install "paramiko<3.4" --force-reinstall

In [4]:
flexmatch_influencer_info, conn_user_info_mtr, conn_user_media_info = get_all_infos()

DB 접속 성공
DB 접속 성공
DB 접속 성공


In [48]:
conn_user_info_mtr['acnt_id'] = conn_user_info_mtr['acnt_id'].astype(str)
conn_user_media_info['acnt_id'] = conn_user_media_info['acnt_id'].astype(str)

conn_user_media_info = conn_user_media_info.drop_duplicates("media_id")

In [5]:
# flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].astype('Int64')
# flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].fillna(0)
# flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].astype(int)
# flexmatch_influencer_info

flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].fillna(0).astype(int)
flexmatch_influencer_info

,user_id,member_uid,add1
0,admin,0,13F80C4C43DC63E772650995CA18639F
1,test901,5180,su.jin.1995
2,samelee47,0,_samelee47
3,tkddmlwns^^^DD,0,oddity
4,ssh8918,0,https://www.instagram.com/flex.item/?hl=
...,...,...,...
1811,dsw01265@naver.com,13632,dearsol_mommy
1812,buffalo37@naver.com,13670,carefulbodyy
1813,easy-@naver.com,13703,easyisnteasy
1814,tedydbs@nate.com,13715,syuni_mom


In [ ]:
flexmatch_influencer_info['add1'] = flexmatch_influencer_info['add1'].str.replace('https://www.instagram.com/', '')
flexmatch_influencer_info['acnt_nm'] = flexmatch_influencer_info['add1'].str.replace('/', '')

In [53]:
new_profile_data = pd.concat([not_conn_user_info_mtr, conn_user_info_mtr], axis=0)
new_media_data = pd.concat([not_conn_user_media_info, conn_user_media_info], axis=0)

In [78]:
new_profile_data['acnt_id'].nunique()

1044

In [ ]:
category_labels = ['IT', '게임', '결혼/연애', '교육', '다이어트/건강보조식품', '만화/애니/툰', '문구/완구', '미술/디자인', '반려동물', '베이비/키즈', '뷰티', '브랜드공식계정',
                    '사진/영상', '셀럽', '스포츠', '시사', '엔터테인먼트', '여행/관광', '유명장소/핫플', '일상', '자동차/모빌리티', '짤/밈', '취미', '패션', '푸드', '홈/리빙']

# 아래 코드 확인 이후에 수정 진행
merged_df, new_merged_df, predict_df = tokenize_and_predict_batch(new_profile_data, new_media_data, category_labels)
new_merged_df = new_merged_df[['acnt_id', 'acnt_nm']].reset_index(drop=True)
predict_df.reset_index(drop=True, inplace=True)

# final data after category labeling
final_predict_df = pd.concat([new_merged_df, predict_df], axis=1)

Map: 100%|██████████| 43007/43007 [00:06<00:00, 7121.90 examples/s]


✅ Using GPU: NVIDIA GeForce RTX 4090


In [172]:
# # new_profile_data 자체랑 new_profile이랑 media랑 합친거(merged_df) 개수가 안맞음
# # 아예 게시물 자체가 없는 사람들
# # 따라서 tokenize_and_predict_batch 함수에서 outer join을 시도

# print(new_merged_df['acnt_id'].nunique())
# lst = new_merged_df['acnt_id'].unique()
# new_profile_data[~new_profile_data['acnt_id'].isin(lst)]

In [159]:
# # outer join 후에 확인 진행
# new_2 = pd.merge(new_profile_data, new_media_data, on='acnt_id', how='outer')
# new_2['acnt_id'].nunique()

In [ ]:
db_merge_df = pd.merge(final_predict_df, flexmatch_influencer_info, on='acnt_nm', how='left')

main_category = db_merge_df.groupby(['acnt_id', 'acnt_nm'])['bert_top_label'].agg(lambda x: x.value_counts().idxmax()).to_frame().reset_index().rename(columns={'bert_top_label' : 'main_category'})

top_3_labels = db_merge_df.groupby(['acnt_id', 'acnt_nm'])['bert_top_label'].value_counts().groupby(level=[0,1]).head(3).reset_index(name='count')
top_3_labels_joined = (top_3_labels.groupby(['acnt_id', 'acnt_nm'])['bert_top_label'].apply(lambda x: '@'.join(x)).reset_index(name='top_3_category'))

top_3_labels_joined = top_3_labels_joined.drop(columns=['acnt_id', 'acnt_nm'])
final_df = pd.concat([main_category, top_3_labels_joined], axis=1)

834


In [186]:
# aa = new_media_data['acnt_id'].unique()
# merged_df[~merged_df['acnt_id'].isin(aa)]['acnt_id'].to_list()

In [ ]:
final_df_list = final_df['acnt_id'].drop_duplicates().to_list()
# merged_df[~merged_df['acnt_id'].isin(final_df_list)].drop_duplicates('acnt_id')[['acnt_id', 'acnt_nm']]

# 인스타그램 게시물이 없거나 게시물이 있어도 글자가 없어서 태깅 안된 사람들
no_category_user_df = merged_df[~merged_df['acnt_id'].isin(final_df_list)].drop_duplicates('acnt_id')[['acnt_id', 'acnt_nm']].reset_index(drop=True)

# 근데 여기서 게시물 자체가 없는 사람은 제외하고 태깅
has_media_user_list = new_media_data['acnt_id'].unique()
no_post_user_list = merged_df[~merged_df['acnt_id'].isin(has_media_user_list)]['acnt_id'].to_list()
no_category_user_df = no_category_user_df[~no_category_user_df['acnt_id'].isin(no_post_user_list)]

no_category_user_df['main_category'] = '일상'
no_category_user_df['top_3_category'] = None

In [184]:
all_final_df = pd.concat([final_df, no_category_user_df], axis=0)
all_final_df.drop_duplicates(['acnt_id'], inplace=True)

In [185]:
all_final_df

,acnt_id,acnt_nm,main_category,top_3_category
0,17841400002751973,licoco___jin,베이비/키즈,베이비/키즈@뷰티
1,17841400011995621,aaari_678,베이비/키즈,베이비/키즈@뷰티@엔터테인먼트
2,17841400038455668,kos9803,여행/관광,여행/관광@유명장소/핫플@베이비/키즈
3,17841400039750009,bichon.mayo,반려동물,반려동물@유명장소/핫플
4,17841400054238696,byulnanmam_official,홈/리빙,홈/리빙@패션@일상
...,...,...,...,...
204,17841465935937693,__moionde,일상,None
206,17841469588134780,review.cat,일상,None
207,17841470887323678,brandlinker.jb,일상,None
208,17841472291806493,lll.ove0103,일상,None
